In [11]:
import requests
from datetime import datetime, timedelta, date
import pandas as pd
import json
import pymssql
import os
from configparser import ConfigParser
import warnings
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [12]:
def renovar_token(client_id, client_secret, token_url):
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    
    # Obtém novo token
    token = oauth.fetch_token(
        token_url=token_url,
        client_id=client_id,
        client_secret=client_secret
    )
    
    return token

# Exemplo de uso
token = renovar_token(
    client_id='terceirizada',
    client_secret='cf86507fdaf05a4f2cda151544513ae769bd305e',
    token_url='https://terceirizada.api.nexti.com/security/oauth/token'
)
tokenaeon = token['access_token']

# Exemplo de uso
token = renovar_token(
    client_id='provac',
    client_secret='4e6b36b10798aaef19e5cb6ec88f295761c80236',
    token_url='https://provac.api.nexti.com/security/oauth/token'
)
tokenprovac = token['access_token']


In [13]:
# PONTO AEON

#config_manager = ConfigManager()
def obter_dados(start_date, end_date):

  url = f"https://terceirizada.api.nexti.com/clockings/start/{start_date}/finish/{end_date}"

  payload = {}
  headers = {
  'authorization': 'Bearer ' + tokenaeon
  }

  response = requests.request("GET", url, headers=headers, data=payload)

  
  return pd.DataFrame(response.json(), dtype=str)

def format_date(date):
    return date.strftime("%d%m%Y%H%M%S")
def obtem_segundos(data_hora_dimep, time_zone):
    # Dicionário com os ajustes de fuso horário
    fusos_horarios = {
        "America/Sao_Paulo": -3,
        "AMERICA/PORTO_VELHO": -4,
        "AMERICA/ARAGUAINA": -3,
        "EUROPE/MADRID": 2,
        "ATLANTIC/STANLEY": -3,
        "AMERICA/BOGOTA": -5,
        "EUROPE/LISBON": 0,
        "AMERICA/LA_PAZ": -4,
        "AMERICA/PORT_OF_SPAIN": -4,
        "AMERICA/ARGENTINA/CORDOBA": -3,
        "AMERICA/ARGENTINA/RIO_GALLEGOS": -3,
        "AMERICA/ARGENTINA/TUCUMAN": -3,
        "AMERICA/ARGENTINA/BUENOS_AIRES": -3,
        "AMERICA/SANTIAGO": -4,
        "AMERICA/GUAYAQUIL": -5,
        "AMERICA/TEGUCIGALPA": -6,
        "AMERICA/MERIDA": -6,
        "AMERICA/MEXICO_CITY": -6,
        "AMERICA/HERMOSILLO": -7,
        "AMERICA/MANAGUA": -6,
        "AMERICA/PANAMA": -5,
        "AMERICA/ASUNCION": -4,
        "AMERICA/LIMA": -5,
        "AFRICA/JOHANNESBURG": 2,
        "EUROPE/WARSAW": 1,
        "AMERICA/SANTO_DOMINGO": -4,
        "AMERICA/MONTERREY": -6,
        "AMERICA/PORT-AU-PRINCE": -5,
        "AMERICA/CHICAGO": -6,
        "EUROPE/BERLIN": 1,
        "EUROPE/AMSTERDAM": 1,
        "AMERICA/MONTEVIDEO": -3,
        "AMERICA/PHOENIX": -7,
        "ASIA/AQTOBE": 5
    }

    # Extrair a hora do registro
    hora = int(data_hora_dimep[8:10])
    minuto = int(data_hora_dimep[10:12])
    
    # Ajustar a hora de acordo com o fuso horário
    ajuste = fusos_horarios.get(time_zone, 0)
    hora_ajustada = hora + ajuste
    
    # Ajustar a data se necessário
    if hora_ajustada < 0:
        hora_ajustada += 24
    elif hora_ajustada >= 24:
        hora_ajustada -= 24
    
    # Calcular os segundos
    segundos = (hora_ajustada * 3600) + (minuto * 60)
    
    return segundos



In [14]:
#PROVAC

# PONTO DA PROVAC
#config_manager = ConfigManager()
def obter_dados_provac(start_date, end_date):

  url = f"https://provac.api.nexti.com/clockings/start/{start_date}/finish/{end_date}"

  payload = {}
  headers = {
  'Authorization': 'Bearer ' + tokenprovac
    }

  response = requests.request("GET", url, headers=headers, data=payload)

  
  return pd.DataFrame(response.json(), dtype=str)

def format_date(date):
    return date.strftime("%d%m%Y%H%M%S")
def obtem_segundos(data_hora_dimep, time_zone):
    # Dicionário com os ajustes de fuso horário
    fusos_horarios = {
        "America/Sao_Paulo": -3,
        "AMERICA/PORTO_VELHO": -4,
        "AMERICA/ARAGUAINA": -3,
        "EUROPE/MADRID": 2,
        "ATLANTIC/STANLEY": -3,
        "AMERICA/BOGOTA": -5,
        "EUROPE/LISBON": 0,
        "AMERICA/LA_PAZ": -4,
        "AMERICA/PORT_OF_SPAIN": -4,
        "AMERICA/ARGENTINA/CORDOBA": -3,
        "AMERICA/ARGENTINA/RIO_GALLEGOS": -3,
        "AMERICA/ARGENTINA/TUCUMAN": -3,
        "AMERICA/ARGENTINA/BUENOS_AIRES": -3,
        "AMERICA/SANTIAGO": -4,
        "AMERICA/GUAYAQUIL": -5,
        "AMERICA/TEGUCIGALPA": -6,
        "AMERICA/MERIDA": -6,
        "AMERICA/MEXICO_CITY": -6,
        "AMERICA/HERMOSILLO": -7,
        "AMERICA/MANAGUA": -6,
        "AMERICA/PANAMA": -5,
        "AMERICA/ASUNCION": -4,
        "AMERICA/LIMA": -5,
        "AFRICA/JOHANNESBURG": 2,
        "EUROPE/WARSAW": 1,
        "AMERICA/SANTO_DOMINGO": -4,
        "AMERICA/MONTERREY": -6,
        "AMERICA/PORT-AU-PRINCE": -5,
        "AMERICA/CHICAGO": -6,
        "EUROPE/BERLIN": 1,
        "EUROPE/AMSTERDAM": 1,
        "AMERICA/MONTEVIDEO": -3,
        "AMERICA/PHOENIX": -7,
        "ASIA/AQTOBE": 5
    }

    # Extrair a hora do registro
    hora = int(data_hora_dimep[8:10])
    minuto = int(data_hora_dimep[10:12])
    
    # Ajustar a hora de acordo com o fuso horário
    ajuste = fusos_horarios.get(time_zone, 0)
    hora_ajustada = hora + ajuste
    
    # Ajustar a data se necessário
    if hora_ajustada < 0:
        hora_ajustada += 24
    elif hora_ajustada >= 24:
        hora_ajustada -= 24
    
    # Calcular os segundos
    segundos = (hora_ajustada * 3600) + (minuto * 60)
    
    return segundos

In [5]:
def obtem_data(data_hora_dimep):
    """
    Converte a string de data do formato DDMMYYYY para um objeto de data Python.
    
    :param data_hora_dimep: String no formato 'DDMMYYYYhhmmss'
    :return: Objeto datetime representando a data
    """
    dia = data_hora_dimep[0:2]
    mes = data_hora_dimep[2:4]
    ano = data_hora_dimep[4:8]
    
    # Cria uma string de data no formato 'DD/MM/YYYY'
    data_formatada = f"{dia}/{mes}/{ano}"
    
    # Converte a string para um objeto datetime
    data_objeto = datetime.strptime(data_formatada, "%d/%m/%Y")
    
    return data_objeto.strftime("%Y%m%d")

def ajusta_id(item_id, tamanho_campo):
    """
    Ajusta o ID para o tamanho do campo, removendo dígitos do início se necessário.
    
    :param item_id: O ID original (pode ser int ou str)
    :param tamanho_campo: O tamanho máximo permitido para o campo
    :return: O ID ajustado (como string)
    """
    # Converte o ID para string
    id_str = str(item_id)
    
    # Verifica se o comprimento do ID é maior que o tamanho do campo
    if len(id_str) > tamanho_campo:
        # Se for maior, pega os últimos dígitos
        return id_str[-tamanho_campo:]
    else:
        # Se não for maior, retorna o ID original
        return id_str


In [15]:
# Data de hoje
data_hoje = datetime.now()-timedelta(1)
print(data_hoje)
# Calculando o mês anterior
mes_anterior = data_hoje.month - 1 if data_hoje.month > 1 else 12
ano_anterior = data_hoje.year if data_hoje.month > 1 else data_hoje.year - 1

# Data do dia 21 do mês anterior
dia_21_mes_anterior = datetime(year=ano_anterior, month=mes_anterior, day=21)


data = pd.date_range(dia_21_mes_anterior, data_hoje)

dftt = pd.DataFrame()

for dt in data:
    # Define datas de início e fim
    start_date = dt
    end_date = datetime(dt.year, dt.month, dt.day, 23, 59, 0)
    
    print(f'Data Inicial: {start_date} e data ,final: {end_date}')
    # Formata datas
    formatted_start = format_date(start_date)
    formatted_end = format_date(end_date)

    # Obtém os dados de clockings
    df = obter_dados(formatted_start, formatted_end)
    df2 = obter_dados_provac(formatted_start, formatted_end)
    
    dftt = pd.concat([dftt, df, df2])

    if dftt is not None:
        print("Dados de clockings obtidos com sucesso.")
    else:
        print("Falha ao obter os dados de clockings.")

novo_df = []
for i, row in dftt.iterrows():
    dData = obtem_data(row["clockingDate"])
    nSegundos = obtem_segundos(row["clockingDate"], row["timezone"])
    cCodNSR = ajusta_id(row["id"], 9)

    novo_registro = [
        row["id"],                    # RR1_VALCON
        row["personPIS"],             # RR1_CODPIS
        row["clockingCollectorName"], # RR1_CODREL
        row["clockingCollectorName"], # RR1_CODREP
        cCodNSR,                      # RR1_CODNSR
        dData,                        # RR1_DATMAR
        nSegundos,                    # RR1_NUMMAR
        "clockinmobile",              # RR1_CODUNI
        "0",                          # RR1_LOGIP
        "0",                          # RR1_GEOFEN
        "99999999999",                # RR1_CCTREP
        0                             # RR1_VRSLAY
    ]
    
    novo_df.append(novo_registro)

2025-01-19 13:10:15.154347
Data Inicial: 2024-12-21 00:00:00 e data ,final: 2024-12-21 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-22 00:00:00 e data ,final: 2024-12-22 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-23 00:00:00 e data ,final: 2024-12-23 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-24 00:00:00 e data ,final: 2024-12-24 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-25 00:00:00 e data ,final: 2024-12-25 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-26 00:00:00 e data ,final: 2024-12-26 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-27 00:00:00 e data ,final: 2024-12-27 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-28 00:00:00 e data ,final: 2024-12-28 23:59:00
Dados de clockings obtidos com sucesso.
Data Inicial: 2024-12-29 00:00:00 e data ,final: 2024-12-29 23:59:00
Dados de clockings obtidos com s

In [17]:
# Criar um novo DataFrame com os dados processados
resultado_df = pd.DataFrame(novo_df, columns=[
    "RR1_VALCON", "RR1_CODPIS", "RR1_CODREL", "RR1_CODREP", "RR1_CODNSR", 
    "RR1_DATMAR", "RR1_NUMMAR", "RR1_CODUNI", "RR1_LOGIP", "RR1_GEOFEN", 
    "RR1_CCTREP", "RR1_VRSLAY"
])
# Verificar se PIS existe
def conect_totvs():
    serv = '192.168.0.236'
    databa = 'PROTHEUS_PRODUCAO'
    user = 'ismael.silva'
    passa = 'w!1zayeUAM'
    conn = pymssql.connect(serv, user, passa, databa)
    cursor = conn.cursor(as_dict=True)
    return conn, cursor

con_tot, cursor = conect_totvs()
query="""
SELECT 
    dbo.SRA010.RA_MAT, 
    dbo.SRA010.RA_NOME,
    dbo.SRA010.R_E_C_N_O_ AS REGISTRO, 
    dbo.SRA010.RA_TNOTRAB, 
    dbo.SRA010.RA_PIS AS RR1_CODPIS
FROM 
    dbo.SRA010
WHERE 
  dbo.SRA010.RA_SITFOLH <> 'D' -- Assumindo que esta é a condição para "não demitido"
  AND dbo.SRA010.D_E_L_E_T_<> '*'
"""
sra = pd.read_sql_query(query, con_tot, dtype=str)
sra['RR1_CODPIS'] = sra['RR1_CODPIS'].str.strip()
sra['existe'] = True
resultado_df2 = pd.merge(resultado_df, sra[['RR1_CODPIS', 'existe']], how='left', on='RR1_CODPIS')


# VERIFICA SE MARCAÇÃO EXISTE
queryrr1="""
select 
    RR1_VALCON, 
    RR1_CODPIS, 
    'True' as Existe_marcacao
from 
    RR1010 
WHERE 
    D_E_L_E_T_ <> '*'
"""
rr1 = pd.read_sql_query(queryrr1, con_tot, dtype=str)
rr1['RR1_VALCON'] = rr1['RR1_VALCON'].str.replace('.0', '')
resultado_df3 = pd.merge(resultado_df2, rr1, how='left', on='RR1_VALCON')
dfMarcacao = resultado_df3[(resultado_df3['existe']==True)&(resultado_df3['RR1_CODPIS_y'].isnull())]
dfMarcacao.drop(columns={'RR1_CODPIS_y'}, inplace=True)
dfMarcacao.rename(columns={'RR1_CODPIS_x':'RR1_CODPIS'}, inplace=True)

In [8]:
print('Quantidade de Marcação pendente')
print(dfMarcacao.groupby('RR1_DATMAR').size())

Quantidade de Marcação pendente
RR1_DATMAR
20250109     11
20250110      4
20250113     17
20250114     71
20250115    159
20250116     98
dtype: int64


In [18]:
def obter_maior_recno(cursor):
    query = "SELECT MAX(R_E_C_N_O_) as max_recno FROM RR1010"
    cursor.execute(query)
    result = cursor.fetchone()
    return result['max_recno'] if result['max_recno'] is not None else 0

def inserir_marcacoes(df_marcacoes, cursor, conn):
    maior_recno = obter_maior_recno(cursor)
    
    for _, row in df_marcacoes.iterrows():
        maior_recno += 1
        query = """
        INSERT INTO RR1010 (
            RR1_VALCON, RR1_CODPIS, RR1_CODREL, RR1_CODREP, RR1_CODNSR, 
            RR1_DATMAR, RR1_NUMMAR, RR1_CODUNI, RR1_LOGIP, RR1_GEOFEN, 
            RR1_CCTREP, RR1_VRSLAY, R_E_C_N_O_
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        values = (
            row['RR1_VALCON'], row['RR1_CODPIS'], row['RR1_CODREL'], row['RR1_CODREP'],
            row['RR1_CODNSR'], row['RR1_DATMAR'], row['RR1_NUMMAR'], row['RR1_CODUNI'],
            row['RR1_LOGIP'], row['RR1_GEOFEN'], row['RR1_CCTREP'], row['RR1_VRSLAY'],
            maior_recno
        )
        try:
            cursor.execute(query, values)
            conn.commit()
            print(f"Marcação inserida com sucesso: {row['RR1_VALCON']} (R_E_C_N_O_: {maior_recno})")
        except Exception as e:
            conn.rollback()
            print(f"Erro ao inserir marcação {row['RR1_VALCON']}: {str(e)}")

In [19]:
# Uso da função
try:
    con_tot, cursor = conect_totvs()
    inserir_marcacoes(dfMarcacao, cursor, con_tot)
finally:
    cursor.close()
    con_tot.close()

Marcação inserida com sucesso: 1553561150 (R_E_C_N_O_: 1272482)
Marcação inserida com sucesso: 1553562298 (R_E_C_N_O_: 1272483)
Marcação inserida com sucesso: 1553562570 (R_E_C_N_O_: 1272484)
Marcação inserida com sucesso: 1553147675 (R_E_C_N_O_: 1272485)
Marcação inserida com sucesso: 1553148205 (R_E_C_N_O_: 1272486)
Marcação inserida com sucesso: 1553563214 (R_E_C_N_O_: 1272487)
Marcação inserida com sucesso: 1550764325 (R_E_C_N_O_: 1272488)
Marcação inserida com sucesso: 1552999449 (R_E_C_N_O_: 1272489)
Marcação inserida com sucesso: 1551945415 (R_E_C_N_O_: 1272490)
Erro ao inserir marcação 1551945415: (2601, b"Cannot insert duplicate key row in object 'dbo.RR1010' with unique index 'RR1010_UNQ'. The duplicate key value is (1.55195e+009, 0).DB-Lib error message 20018, severity 14:\nGeneral SQL Server error: Check messages from the SQL Server\n")
Marcação inserida com sucesso: 1551945435 (R_E_C_N_O_: 1272492)
Marcação inserida com sucesso: 1551945470 (R_E_C_N_O_: 1272493)
Marcação in